In [42]:
import org.apache.spark.ml.linalg. DenseVector
import org.apache.spark.sql.types.IntegerType
import org.apache.spark.sql.functions.{col, round, udf}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [2]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val df_train = spark.read.options(csvOptions).csv("/labs/slaba05/lab05_train.csv").drop("_c0").cache()
val df_test = spark.read.options(csvOptions).csv("/labs/slaba05/lab05_test.csv").drop("_c0").cache()

Waiting for a Spark session to start...

csvOptions = Map(header -> true, inferSchema -> true)
df_train = [ID: int, CR_PROD_CNT_IL: int ... 114 more fields]
df_test = [ID: int, CR_PROD_CNT_IL: int ... 113 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 113 more fields]

In [3]:
val col_list = df_test.dtypes

col_list = Array((ID,IntegerType), (CR_PROD_CNT_IL,IntegerType), (AMOUNT_RUB_CLO_PRC,DoubleType), (PRC_ACCEPTS_A_EMAIL_LINK,DoubleType), (APP_REGISTR_RGN_CODE,DoubleType), (PRC_ACCEPTS_A_POS,DoubleType), (PRC_ACCEPTS_A_TK,DoubleType), (TURNOVER_DYNAMIC_IL_1M,DoubleType), (CNT_TRAN_AUT_TENDENCY1M,DoubleType), (SUM_TRAN_AUT_TENDENCY1M,DoubleType), (AMOUNT_RUB_SUP_PRC,DoubleType), (PRC_ACCEPTS_A_AMOBILE,DoubleType), (SUM_TRAN_AUT_TENDENCY3M,DoubleType), (CLNT_TRUST_RELATION,StringType), (PRC_ACCEPTS_TK,DoubleType), (PRC_ACCEPTS_A_MTP,DoubleType), (REST_DYNAMIC_FDEP_1M,DoubleType), (CNT_TRAN_AUT_TENDENCY3M,DoubleType), (CNT_ACCEPTS_TK,DoubleType), (APP_MARITAL_STATUS,StringType), (REST_DYNAMIC_SAVE_3M,DoubleType), (CR_PROD_CNT_VCU,IntegerType), (REST_AVG_CUR,DoubleT...


Array((ID,IntegerType), (CR_PROD_CNT_IL,IntegerType), (AMOUNT_RUB_CLO_PRC,DoubleType), (PRC_ACCEPTS_A_EMAIL_LINK,DoubleType), (APP_REGISTR_RGN_CODE,DoubleType), (PRC_ACCEPTS_A_POS,DoubleType), (PRC_ACCEPTS_A_TK,DoubleType), (TURNOVER_DYNAMIC_IL_1M,DoubleType), (CNT_TRAN_AUT_TENDENCY1M,DoubleType), (SUM_TRAN_AUT_TENDENCY1M,DoubleType), (AMOUNT_RUB_SUP_PRC,DoubleType), (PRC_ACCEPTS_A_AMOBILE,DoubleType), (SUM_TRAN_AUT_TENDENCY3M,DoubleType), (CLNT_TRUST_RELATION,StringType), (PRC_ACCEPTS_TK,DoubleType), (PRC_ACCEPTS_A_MTP,DoubleType), (REST_DYNAMIC_FDEP_1M,DoubleType), (CNT_TRAN_AUT_TENDENCY3M,DoubleType), (CNT_ACCEPTS_TK,DoubleType), (APP_MARITAL_STATUS,StringType), (REST_DYNAMIC_SAVE_3M,DoubleType), (CR_PROD_CNT_VCU,IntegerType), (REST_AVG_CUR,DoubleT...

In [4]:
val df_train_cast = (df_train.columns.toBuffer --= Array("ID", "TARGET")).foldLeft(df_train)((current, c) =>
                                              current.withColumn(c, col(c).cast("float")))
val df_test_cast = (df_test.columns.toBuffer --= Array("ID", "TARGET")).foldLeft(df_test)((current, c) =>
                                              current.withColumn(c, col(c).cast("float")))

df_train_cast = [ID: int, CR_PROD_CNT_IL: float ... 114 more fields]
df_test_cast = [ID: int, CR_PROD_CNT_IL: float ... 113 more fields]


[ID: int, CR_PROD_CNT_IL: float ... 113 more fields]

In [5]:
val df_train2 = df_train_cast.na.fill(0)
val df_test2 = df_test_cast.na.fill(0)

df_train2 = [ID: int, CR_PROD_CNT_IL: float ... 114 more fields]
df_test2 = [ID: int, CR_PROD_CNT_IL: float ... 113 more fields]


[ID: int, CR_PROD_CNT_IL: float ... 113 more fields]

In [6]:
val col_list = df_test_cast.columns.filter(_ != "ID")

col_list = Array(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, AMOUNT_RUB_NAS_PRC, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, TRANS_COUNT_SUP_PRC, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, APP_TRAVEL_PASS, CNT_ACCEPTS_...


Array(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, CLNT_TRUST_RELATION, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, APP_MARITAL_STATUS, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, AMOUNT_RUB_NAS_PRC, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, TRANS_COUNT_SUP_PRC, APP_EDUCATION, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, APP_TRAVEL_PASS, CNT_ACCEPTS_...

In [7]:
val assembler = new VectorAssembler()
                       .setInputCols(col_list)
                       .setOutputCol("features")

assembler = vecAssembler_37d633c151a3


vecAssembler_37d633c151a3

In [8]:
val gbt = new GBTClassifier()
                .setFeaturesCol("features")
                .setLabelCol("TARGET")

gbt = gbtc_5ee9ddb704e4


gbtc_5ee9ddb704e4

In [9]:
val pipeline = new Pipeline()
                  .setStages(Array(assembler,gbt))

pipeline = pipeline_4822b3c8bd43


pipeline_4822b3c8bd43

In [10]:
val pipeline_model = pipeline.fit(df_train2)

pipeline_model = pipeline_4822b3c8bd43


pipeline_4822b3c8bd43

In [11]:
val prediction = pipeline_model.transform(df_test2)

prediction = [ID: int, CR_PROD_CNT_IL: float ... 117 more fields]


[ID: int, CR_PROD_CNT_IL: float ... 117 more fields]

In [32]:
val result = prediction.select("ID", "probability")
                       .withColumnRenamed("ID","id")
                       .repartition(1)

result = [id: int, probability: vector]


lastException: Throwable = null


[id: int, probability: vector]

In [57]:
def toArray: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrayUdf = udf(toArray)
val getValue = udf((arr: Seq[Double]) => arr(1))

toArray = > Array[Double]
toArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)
headValue = UserDefinedFunction(<function1>,DoubleType,Some(List(ArrayType(DoubleType,false))))


lastException: Throwable = null


UserDefinedFunction(<function1>,DoubleType,Some(List(ArrayType(DoubleType,false))))

In [58]:
val result2 = result.withColumn("target", toArrayUdf(col("probability")))
                    .withColumn("target", getValue(col("target")))
                    .drop("probability")

result2 = [id: int, target: double]


[id: int, target: double]

In [59]:
result2.write.option("header",true)  
            .option("delimiter","\t")        
            .format("csv").save("Lab05.csv")